In [28]:
from kidney_disease.entity.config_entity import DataIngestionConfig
import pandas as pd
from kidney_disease.entity.config_entity import DataIngestionConfig, DataTransformationConfig,DataValidationConfig,   \
ModelTrainerConfig,ModelEvaluationConfig,ModelPusherConfig,TrainingPipelineConfig
from kidney_disease.entity import *
from kidney_disease.util.util import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from category_encoders.binary import BinaryEncoder

In [117]:

schema_file_path = 'F:\chronic_disease\config\schema.yaml'

schema_df = read_yaml_file(file_path=schema_file_path)

numerical_columns = schema_df[NUMERICAL_COLUMN_KEY]
categorical_columns = schema_df[CATEGORICAL_COLUMN_KEY]



cat_pipeline = Pipeline(steps=[('binaryencoding', BinaryEncoder()), ("impute", KNNImputer(n_neighbors=2)) ])
num_pipeline = Pipeline(steps=[('knn_imputer', KNNImputer(n_neighbors=3)), ('scaler', StandardScaler())])



In [118]:
df = 'https://raw.githubusercontent.com/Adityashinde1/chronic_disease/main/data/kidney_disease.csv'
df = pd.read_csv(df)

In [119]:
df['pcv'] = df['pcv'].replace("?", np.NaN)
df['wc'] = df['wc'].replace("?", np.NaN)
df['rc'] = df['rc'].replace("?", np.NaN)

df = df.drop('id',axis=1)


rc_column_value = 5.1
pcv_column_value = 27
wc_column_value = 8500

df['rc'] = df['rc'].fillna(value=rc_column_value)
df['pcv'] = df['pcv'].fillna(value=pcv_column_value)
df['wc'] = df['wc'].fillna(value=wc_column_value)

df = df.astype({"rc": float, "pcv": int, "wc": int})

In [126]:
preprocessing = ColumnTransformer([
                ('num_pipeline', num_pipeline, numerical_columns),
                ('cat_pipeline', cat_pipeline, categorical_columns)
                ])

In [128]:
train_set, test_set = train_test_split(df, test_size=0.2)
print(train_set.shape)
print(test_set.shape)

(320, 25)
(80, 25)


In [129]:

preprocessing.fit_transform(train_set)

array([[ 0.54599095, -0.42592618,  1.41899496, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.11215137, -0.42592618, -1.28743251, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.94230324, -0.42592618, -0.38529002, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-1.03925821,  0.29142318,  1.41899496, ...,  1.        ,
         0.        ,  1.        ],
       [-1.60541863, -1.14327554,  0.51685247, ...,  1.        ,
         0.        ,  1.        ],
       [-2.05834696,  0.29142318,  1.41899496, ...,  1.        ,
         0.        ,  1.        ]])